In [ ]:
 # Step 1: Remove old SadTalker if it exists
import os, shutil
if os.path.exists("SadTalker"):
    print("Removing existing SadTalker folder...")
    shutil.rmtree("SadTalker")

# Step 2: Clone SadTalker repo
print("Cloning SadTalker repo...")
!git clone https://github.com/OpenTalker/SadTalker.git

# Step 3: Move into SadTalker folder
%cd SadTalker

# Modify requirements.txt to remove specific torch version
print("Modifying requirements.txt to remove specific torch version...")
!sed -i 's/torch==1.12.1//g' requirements.txt
!sed -i 's/torchvision==0.13.1//g' requirements.txt
!sed -i 's/torchaudio==0.12.1//g' requirements.txt

# Step 4: Install SadTalker's requirements
print("Installing SadTalker requirements...")
!pip install -r requirements.txt

# Step 5: Install Coqui TTS (used for voice synthesis)
print("Installing TTS package...")
!pip install TTS

# Step 6: Download pretrained models
print("Downloading SadTalker models...")
!bash scripts/download_models.sh

# Step 7: Install extra video/audio dependencies
print("Installing video processing tools...")
!pip install ffmpeg-python imageio imageio-ffmpeg
!apt-get update -y && apt-get install -y ffmpeg
!pip install --upgrade imageio

# Step 8: Go back to root if needed
%cd ..

# Step 9: Import and check everything is working
print("Verifying installations...")
import numpy as np
import torch
import torchvision
import TTS

print("✅ All core libraries imported successfully!")

# Optional: Show installed versions
!pip show torch
!pip show torchvision
!pip show torchaudio
!pip show numpy
!pip show TTS
!pip show basicsr || echo "basicsr not found"
!pip show facexlib || echo "facexlib not found"
!pip show gfpgan || echo "gfpgan not found"

In [ ]:
import fileinput
import sys

file_path = '/content/SadTalker/src/face3d/util/my_awing_arch.py'
old_string = 'preds = preds.astype(np.float, copy=False)'
new_string = 'preds = preds.astype(float, copy=False)'  # Use built-in float

# Edit the file in place to replace deprecated np.float
for line in fileinput.FileInput(file_path, inplace=True):
    if old_string in line:
        line = line.replace(old_string, new_string)
    sys.stdout.write(line)

print(f"✅ Fixed np.float issue in: {file_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/SadTalker/src/face3d/util/my_awing_arch.py' -> '/content/SadTalker/src/face3d/util/my_awing_arch.py.bak'

In [ ]:
import sys
import os
import numpy as np

file_path = '/content/SadTalker/src/face3d/util/preprocess.py'
debug_string_identifier = "DEBUG: align_img vars -"

try:
    # Read the current content
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # Check if imports are already there
    import_sys_present = any("import sys" in line for line in lines)
    import_numpy_present = any("import numpy" in line for line in lines)

    updated_lines = []

    # Add imports if missing
    if not import_sys_present:
        updated_lines.append("import sys\n")
    if not import_numpy_present:
        updated_lines.append("import numpy as np\n")

    # Modify content
    for line in lines:
        if debug_string_identifier in line:
            continue  # Remove debug line
        elif 'trans_params = np.array([w0, h0, s, t[0], t[1]])' in line:
            updated_lines.append('    trans_params = np.array([w0, h0, s, t[0][0], t[1][0]])  # Fixed shape error\n')
        else:
            updated_lines.append(line)

    # Write back
    with open(file_path, 'w') as f:
        f.writelines(updated_lines)

    print(f"✅ Patch applied to '{file_path}' successfully!")

except FileNotFoundError:
    print(f"❌ File not found at {file_path}. Make sure SadTalker is cloned properly.")
except Exception as e:
    print(f"⚠️ Error occurred during patching: {e}")


In [ ]:
import fileinput
import sys
import os

# Path to the problematic file inside basicsr
basicsr_file_path = '/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py'
problematic_import = 'from torchvision.transforms.functional_tensor import rgb_to_grayscale'

print(f"🚧 Attempting to patch '{basicsr_file_path}' to remove the broken import...")

try:
    if not os.path.exists(basicsr_file_path):
        print(f"❌ File not found: {basicsr_file_path} — basicsr might not be installed yet.")
    else:
        for line in fileinput.FileInput(basicsr_file_path, inplace=True):
            if problematic_import in line:
                print(f"⚠️ Removed line: {line.strip()}", file=sys.stderr)  # Logs removed line
                continue  # Skip writing the problematic line
            sys.stdout.write(line)

        print(f"✅ Successfully patched '{basicsr_file_path}' and removed the import.")

except Exception as e:
    print(f"🔥 Error during patching: {e}")


In [ ]:
from google.colab import files

# Prompt the user to upload an image (preferably a clear front-facing face)
print("📷 Upload a portrait image for the avatar (JPG or PNG, front-facing preferred)")
uploaded = files.upload()  # Opens file picker

# Grab the uploaded file name
import os
filename = next(iter(uploaded))
print(f"✅ Uploaded: {filename}")

# You can now use this `filename` as input for SadTalker or image processing


In [ ]:
# 🔊 This is the sample script your avatar will speak.
# You can change it to anything depending on the lesson topic.
lesson_text = """
Hello students! Today, I will teach you about the process of photosynthesis.
Photosynthesis is how plants convert sunlight, water, and carbon dioxide into energy and oxygen.
Let's explore how this amazing process works!
"""

# 🧠 Tip for your team:
# - Keep it under 2–3 lines for faster processing and clarity
# - Keep language simple so the avatar syncs lips better


In [ ]:
# Re-install Coqui TTS to ensure it's available
print("Re-installing TTS package...")
!pip install TTS
print("✅ TTS package re-installed successfully.")

In [ ]:
import os
from TTS.api import TTS

# 🎙️ Load a pre-trained English TTS model (Tacotron2 with DDC)
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=False)

# 📂 Output filename for the generated audio
output_wav = "lesson.wav"

# 🧠 Convert the lesson_text into a speech audio file
tts.tts_to_file(text=lesson_text, file_path=output_wav)

print(f"✅ Audio file generated: {output_wav}")

In [ ]:
import os
import shutil

# 📷 Automatically find the uploaded avatar image file (jpg/png/jpeg)
avatar_image = [f for f in os.listdir() if f.lower().endswith(('.png', '.jpg', '.jpeg'))][0]

# 🔉 Path to the generated lesson audio
audio_path = output_wav  # Must be defined from the previous TTS step

# 📁 Create input/output folders if not already there
os.makedirs('input', exist_ok=True)
os.makedirs('output', exist_ok=True)

# 🗂️ Move files into input folder for SadTalker
shutil.copy(avatar_image, 'input/avatar.png')
shutil.copy(audio_path, 'input/lesson.wav')

# 📍 Make sure you're inside SadTalker directory before running inference
current_dir = os.getcwd()
if os.path.basename(current_dir) != 'SadTalker':
    %cd SadTalker

# 🚀 Run SadTalker to generate the talking avatar
# Added --size 256 and removed --enhancer gfpgan for faster processing
print("🔥 Running SadTalker inference with faster settings...")
!python inference.py \
  --driven_audio ../input/lesson.wav \
  --source_image ../input/avatar.png \
  --result_dir ../output \
  --still \
  --preprocess full \
  --size 256 \
  --batch_size 1 \
  --expression_scale 1.0 \
  --input_yaw 0 --input_pitch 0 --input_roll 0


# ⬅️ Go back to root directory after running the model
if os.path.basename(os.getcwd()) == 'SadTalker':
    %cd ..

In [ ]:
from IPython.display import Video
import os

# 📂 Define the output directory
output_dir = 'output'

# 🎥 Find any MP4 file generated by SadTalker
mp4_files = [f for f in os.listdir(output_dir) if f.endswith('.mp4')]

if mp4_files:
    video_file = mp4_files[0]
    video_path = f"{output_dir}/{video_file}"
    print(f"✅ Found video file: {video_path}")

    # 🎬 Display the video right in the notebook
    print("🎥 Playing generated avatar video below...")
    display(Video(video_path, embed=True, width=400))
else:
    print(f"❌ No video found in '{output_dir}' folder.")
    print("⚠️ Check if SadTalker ran successfully and generated the output.")
